In [ ]:
# only run this on Google Colab
!pip install geopandas
!pip install folium

In [37]:
import pandas as pd
import seaborn as sns
import folium
import numpy as np
import matplotlib.pyplot as plt

# Load

In [38]:
main_df = pd.read_csv('https://raw.githubusercontent.com/Dynovski/BESTHL-AI/main/train_data.csv?token=GHSAT0AAAAAABTLWN7I6LMAI3BK5ZSQSE7WYSZQ4DA', error_bad_lines=False)
print(main_df.head())

C:\Users\Maciek\AppData\Local\Temp\ipykernel_14468\363779577.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  main_df = pd.read_csv('https://raw.githubusercontent.com/Dynovski/BESTHL-AI/main/train_data.csv?token=GHSAT0AAAAAABTLWN7I6LMAI3BK5ZSQSE7WYSZQ4DA', error_bad_lines=False)


         BBLE  BORO  BLOCK   LOT EASEMENT                OWNER BLDGCL  \
0  1001791101     1    179  1101      NaN  155 FRANKLIN ST LLC     RK   
1  1000163436     1     16  3436      NaN         BROADLAR LLC     R4   
2  1000761255     1     76  1255      NaN         HAM, RICHARD     R4   
3  1000161179     1     16  1179      NaN        SHIM, HAE JIN     R4   
4  1001751526     1    175  1526      NaN        BRIT GIRL LLC     R4   

  TAXCLASS  LTFRONT  LTDEPTH  ... VALTYPE    Borough   Latitude  Longitude  \
0        4        0        0  ...   AC-TR  MANHATTAN  40.719232 -74.008095   
1        2        0        0  ...   AC-TR  MANHATTAN  40.709270 -74.016487   
2        2        0        0  ...   AC-TR  MANHATTAN  40.708006 -74.006164   
3        2        0        0  ...   AC-TR  MANHATTAN  40.709506 -74.018002   
4        2        0        0  ...   AC-TR  MANHATTAN  40.718024 -74.003903   

   Community Board  Council District  Census Tract        BIN  \
0            101.0         

b'Skipping line 5856: expected 40 fields, saw 70\nSkipping line 7939: expected 40 fields, saw 49\n'


In [ ]:
processed_df = pd.read_csv('processed.csv')
processed_df.head()

# Basic stuff

In [ ]:
main_df.shape

In [ ]:
main_df.columns

In [ ]:
main_df.info()

In [ ]:
main_df.describe()

In [ ]:
main_df['POSTCODE'].value_counts()

In [ ]:
sns.scatterplot(data=main_df, x=main_df.index, y=main_df["FULLVAL"])

In [ ]:
type(main_df['TAXCLASS'][0])

In [ ]:
main_df['TAXCLASS'].head(19)

In [ ]:
sns.catplot(data=main_df, x=main_df.index, y=main_df["TAXCLASS"])

# Map

In [ ]:
main_df['Latitude'].value_counts().head(30)

In [ ]:
main_df['Longitude'].value_counts().head(30)

In [24]:
# Watch out! sampling resets index
map_df = main_df[main_df['Latitude'].notna()].sample(frac=1).reset_index(drop=True)

# using too big location_number param results in slow map 
location_number = 1000

map_df = map_df[map_df['Longitude'].notna()].head(location_number)
print(map_df.shape)

(1000, 40)


In [ ]:
map_df['Latitude'].loc[0]
type(map_df['Latitude'].loc[0])

In [ ]:
map_df['Longitude'].loc[0]
type(map_df['Latitude'].loc[0])

In [ ]:
map_full = folium.Map(location=[40.719232, -74.00809], zoom_start=9)

In [ ]:
for index, _ in map_df.iterrows():
    folium.Marker([map_df['Latitude'].loc[index], map_df['Longitude'].loc[index]]).add_to(map_full)

In [ ]:
map_full

## Borough clustering

In [39]:
main_df['Borough'].value_counts()

STATEN IS    3775
MANHATTAN    2065
QUEENS       2060
BROOKLYN     1808
BRONX        1806
Name: Borough, dtype: int64

In [47]:
borough_df['FULLVAL mean'] = main_df[['Borough', 'FULLVAL']].groupby(['Borough']).mean()
print(borough_df)

                FULLVAL        Colour  FULLVAL mean
Borough                                            
BRONX      1.404000e+06        [Blue]  1.404000e+06
BROOKLYN   1.627501e+06      [Yellow]  1.627501e+06
MANHATTAN  3.027498e+06      [Yellow]  3.027498e+06
QUEENS     6.826519e+05  [Red, Green]  6.826519e+05
STATEN IS  6.175162e+05      [Purple]  6.175162e+05


In [48]:
borough_df['Colour'] = [['Blue'], ['Yellow'], ['Yellow'], ['Red', 'Green'], ['Purple']]
borough_df.drop('FULLVAL', axis=1, inplace=True)
print(borough_df)

                 Colour  FULLVAL mean
Borough                              
BRONX            [Blue]  1.404000e+06
BROOKLYN       [Yellow]  1.627501e+06
MANHATTAN      [Yellow]  3.027498e+06
QUEENS     [Red, Green]  6.826519e+05
STATEN IS      [Purple]  6.175162e+05


## Separating clusters

### k-means

In [ ]:
from scipy.cluster.vq import kmeans2, whiten

In [ ]:
map_kmeans = folium.Map(location=[40.719232, -74.00809], zoom_start=9)

In [ ]:
coordinates_kmeans = [[map_df['Latitude'].loc[index], map_df['Longitude'].loc[index]] for index, _ in map_df.iterrows()]
coordinates_kmeans = np.array(coordinates_kmeans)
print(type(coordinates_kmeans))
print(len(coordinates_kmeans))
print(coordinates_kmeans[19])
print(type(coordinates_kmeans[19]))

In [ ]:
# non-pythonic :-(
num_clusters_kmeans = set(y)

for idx, place in enumerate(coordinates_kmeans):
    if y[idx] == 0:
        folium.Marker([place[0], place[1]], icon=folium.Icon(color="purple")).add_to(map_kmeans)
    if y[idx] == 1:
        folium.Marker([place[0], place[1]], icon=folium.Icon(color="gray")).add_to(map_kmeans)
    if y[idx] == 2:
        folium.Marker([place[0], place[1]], icon=folium.Icon(color="green")).add_to(map_kmeans)
    if y[idx] == 3:
        folium.Marker([place[0], place[1]], icon=folium.Icon(color="blue")).add_to(map_kmeans)
    if y[idx] == 4:
        folium.Marker([place[0], place[1]], icon=folium.Icon(color="red")).add_to(map_kmeans)

In [ ]:
x, y = kmeans2(whiten(coordinates_kmeans), 5, iter = 20)  
# print(y)
print(coordinates_kmeans[0])
plt.scatter(coordinates_kmeans[:,0], coordinates_kmeans[:,1], c=y)
plt.show()

In [ ]:
map_kmeans

### DBSCAN

In [25]:
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
from shapely.geometry import MultiPoint

In [26]:
coordinates_dbscan = map_df[['Latitude', 'Longitude']].to_numpy()

In [27]:
kms_per_radian = 6371.0088
epsilon = 1.7 / kms_per_radian
db = DBSCAN(eps=epsilon, min_samples=10, algorithm='ball_tree', metric='haversine').fit(np.radians(coordinates_dbscan))
cluster_labels = db.labels_
num_clusters = len(set(cluster_labels))
clusters = pd.Series([coordinates_dbscan[cluster_labels == n] for n in range(num_clusters)])
print('Number of clusters: {}'.format(num_clusters))

Number of clusters: 6


In [28]:
print(type(clusters[2]))
print(len(clusters[2]))
print(clusters[2].shape)

<class 'numpy.ndarray'>
286
(286, 2)


In [29]:
map_dbscan = folium.Map(location=[40.719232, -74.00809], zoom_start=9)

In [30]:
for place in clusters[0]:
    folium.Marker([place[0], place[1]], icon=folium.Icon(color="orange")).add_to(map_dbscan)

print(clusters[0].shape)

(84, 2)


In [31]:
for place in clusters[1]:
    folium.Marker([place[0], place[1]], icon=folium.Icon(color="blue")).add_to(map_dbscan)
    
print(clusters[1].shape)

(347, 2)


In [32]:
for place in clusters[2]:
    folium.Marker([place[0], place[1]], icon=folium.Icon(color="purple")).add_to(map_dbscan)
    
print(clusters[2].shape)

(286, 2)


In [33]:
for place in clusters[3]:
    folium.Marker([place[0], place[1]], icon=folium.Icon(color="green")).add_to(map_dbscan)

print(clusters[3].shape)

(174, 2)


In [34]:
for place in clusters[4]:
    folium.Marker([place[0], place[1]], icon=folium.Icon(color="red")).add_to(map_dbscan)
    
print(clusters[4].shape)

(108, 2)


In [35]:
for place in clusters[5]:
    folium.Marker([place[0], place[1]], icon=folium.Icon(color="gray")).add_to(map_dbscan)

print(clusters[5].shape)

(0, 2)


In [36]:
map_dbscan

# Augment

In [ ]:
augmented_df = processed_df

In [ ]:
augmented_df.loc[(augmented_df['Latitude'] <= 90) & (augmented_df['Latitude'] >= -90)]

In [ ]:
augmented_df.loc[(augmented_df['Longitude'] <= 180) & (augmented_df['Longitude'] >= -180)]

In [ ]:
augmented_df['1-closest neighbour value'] = 0
augmented_df['2-closest neighbour value'] = 0
augmented_df['3-closest neighbour value'] = 0
augmented_df['4-closest neighbour value'] = 0
augmented_df['5-closest neighbour value'] = 0

augmented_df[['1-closest neighbour value', 
              '2-closest neighbour value', 
              '3-closest neighbour value', 
              '4-closest neighbour value', 
              '5-closest neighbour value']].head()

In [ ]:
from sklearn.neighbors import BallTree

In [ ]:
print(augmented_df[['Latitude', 'Longitude']].values.shape)

In [ ]:
tree = BallTree(np.deg2rad(augmented_df[['Latitude', 'Longitude']].values), metric='haversine')

In [ ]:
query_lats = augmented_df['Latitude']
query_lons = augmented_df['Longitude']
print(query_lats.shape)
print(query_lons.shape)
print(query_lats.head())
print(query_lons.head())

In [ ]:
# Find closest city in reference dataset for each in df_other
# use k = 6 for 5 closest neighbors + self
distances, indices = tree.query(np.deg2rad(np.c_[query_lats, query_lons]), k = 6)
# print(len(distances))
# print(len(indices))

r_km = 6371 # multiplier to convert to km (from unit distance)
for name, d, ind in zip(augmented_df.index, distances, indices):
    print(f"Property with index {name} closest matches:")
    for i, index in enumerate(ind):
        print(f"\t{index} with distance {d[i]*r_km:.4f} km")

In [ ]:
distances, indices = tree.query(np.deg2rad(np.c_[query_lats, query_lons]), k = 6)
helper_df = augmented_df['FULLVAL']

r_km = 6371 # multiplier to convert to km (from unit distance)
for work_index, d, ind in zip(augmented_df.index, distances, indices):
    for i, index in enumerate(ind):
        if i:
            augmented_df.loc[work_index, f'{i}-closest neighbour value'] = helper_df.loc[index]

In [ ]:
augmented_df[['1-closest neighbour value', 
              '2-closest neighbour value', 
              '3-closest neighbour value', 
              '4-closest neighbour value', 
              '5-closest neighbour value']].head()

In [ ]:
print(augmented_df.shape)
print(augmented_df.columns)

In [ ]:
augmented_df.to_csv('./augmented.csv', index=False)

In [ ]:
test_df = pd.read_csv('augmented.csv')
print(test_df.shape)
print(test_df.columns)